In [13]:
import os
from urllib import request
from bs4 import BeautifulSoup 
import time
import gzip
import numpy as np
import pandas
from pprint import pprint
data_dir = '/home/idies/workspace/Temporary/raddick/cra_scratch_final/'
jobs_dir = data_dir + 'lodes_wac/'
baltimore_dir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/baltimore/'
g = 0
print('Done!')
os.getcwd()

Done!


'/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act'

s = time.time()
y = os.listdir(data_dir)
hasdir = [x for x in y if x=='lodes_wac']
if (len(hasdir) == 0):
    os.mkdir(jobs_dir)
e = time.time()
g = g + (e-s)
print('Done in {0:,.3f} seconds!'.format(e-s))

# Download WAC data from census.gov

s = time.time()
theurl = 'https://lehd.ces.census.gov/data/lodes/LODES7/'
page = request.urlopen(theurl)
soup = BeautifulSoup(page, 'html.parser')
page.close()

bigtable = soup.find('table')
allstaterows = bigtable.findAll('tr')
#for i in range(7, 61):
for i in range(59, 61):
#for i in range(27, 28):
    allstatecells = allstaterows[i].findAll('td')
    this_state_url = theurl + allstatecells[1].find('a').get('href')+'wac/'
    print(this_state_url)
    #print(this_state_url)
    statepage = request.urlopen(this_state_url)
    statesoup = BeautifulSoup(statepage, 'html.parser')
    statepage.close()
    statetable = statesoup.find('table')
    filerows = statetable.findAll('tr')
    print('Parsing state {0:,.0f} of {1:,.0f}... Downloading {2:,.0f} files...'.format(i, 60, len(filerows)-4))
    for j in range(3, len(filerows)-1):
        filecells = filerows[j].findAll('td')
        this_file_url = this_state_url + filecells[1].find('a').get('href')
        this_file_name = jobs_dir + filecells[1].text
        with open(this_file_name, 'wb') as f:
            r = request.urlopen(this_file_url)
            f.write(r.read())
            r.close()
print('done')            
nFiles = len(os.listdir(jobs_dir))
e = time.time()
g = g + (e-s)
print('Done: downloaded {0:,.0f} files in {1:,.0f} minutes {2:,.0f} seconds!'.format(nFiles, np.floor((e-s)/60), (e-s)%60))

#filename = metadata_dir+'2010_5yr_Summary_FileTemplates.zip'
#print('reading contents of gzip file...')
#with gzip.open(filename, 'rb') as f:
#    file_content = f.read()
#print('Contents of gzip file have been read...')


# Un-gzip

s = time.time()
#states = ['tn', 'tx', 'ut', 'va', 'vt']
states = ['wa', 'wi', 'wv', 'wy']
gzipfiles = []
for this_state in states:
    gzipfiles += [jobs_dir+x for x in os.listdir(jobs_dir) if ('{0:}_wac'.format(this_state) in x) and (x[-3:] == '.gz')]
print('Found {0:,.0f} gzip files...'.format(len(gzipfiles)))

for i in range(0, len(gzipfiles)):
    if ((np.mod(i, 25) == 0) or (i == len(gzipfiles) - 1)):
        print('Un-gzipping file {0:.0f} of {1:.0f}...'.format(i, len(gzipfiles)-1))
    csvfilename = gzipfiles[i][:-3]
    with gzip.open(gzipfiles[i], 'rb') as f:
        file_content = f.read()
    with open(csvfilename, 'wb') as f:
        f.write(file_content)
e = time.time()
g = g + (e-s)
print('Files unzipped in {0:,.0f} seconds!'.format(e-s))


# Delete .gz files

gzfiles = [jobs_dir+x for x in os.listdir(jobs_dir) if x[-3:] == '.gz']
for y in gzfiles:
    os.remove(y)
print('Deleted all .gz files!')

## Parse all CSVs into a single file per state

In [8]:
states = ['nm']
s = time.time()
for this_state in states:
    print('Processing {0:}...'.format(this_state))
    jobs_df = pandas.DataFrame()
    csvfiles = [jobs_dir+x for x in os.listdir(jobs_dir) if ('{0:}_wac'.format(this_state) in x) and (x[-4:] == '.csv')]
    if (this_state == 'in'):
        startfile = 1
    else:
        startfile = 0
    for i in range(startfile, len(csvfiles)):
        if ((np.mod(i, 25) == 0) or (i == len(csvfiles) - 1)):
            print('Importing file {0:,.0f} of {1:,.0f}...'.format(i, len(csvfiles)-1))
        thisyear = int(csvfiles[i][-8:-4])
        xdf = pandas.read_csv(csvfiles[i], low_memory=False)
        xdf = xdf.assign(year = thisyear)
        jobs_df = jobs_df.append(xdf)
    jobs_df = jobs_df.reset_index(drop=True)
    jobs_df.index.name = 'rownumber'
    print('writing outfile...')
    jobs_df.to_csv(jobs_dir+'jobs_data_{0:}.csv'.format(this_state), encoding='utf-8')
e = time.time()
g = g + (e-s)
print('Wrote {0:,.0f} rows in {1:,.0f} minutes {2:,.0f} seconds!'.format(len(jobs_df), np.floor((e-s)/60), (e-s) % 60))

Processing nm...
Importing file 0 of 679...
Importing file 25 of 679...
Importing file 50 of 679...
Importing file 75 of 679...
Importing file 100 of 679...
Importing file 125 of 679...
Importing file 150 of 679...
Importing file 175 of 679...
Importing file 200 of 679...
Importing file 225 of 679...
Importing file 250 of 679...
Importing file 275 of 679...
Importing file 300 of 679...
Importing file 325 of 679...
Importing file 350 of 679...
Importing file 375 of 679...
Importing file 400 of 679...
Importing file 425 of 679...
Importing file 450 of 679...
Importing file 475 of 679...
Importing file 500 of 679...
Importing file 525 of 679...
Importing file 550 of 679...
Importing file 575 of 679...
Importing file 600 of 679...
Importing file 625 of 679...
Importing file 650 of 679...
Importing file 675 of 679...
Importing file 679 of 679...
writing outfile...
Wrote 5,980,252 rows in 30 minutes 47 seconds!


# Get Baltimore data and write it out

s = time.time()
df = pandas.read_csv(jobs_dir+'jobs_data_{0:}.csv'.format(this_state), encoding='utf-8', low_memory=False, index_col='rownumber')
#df = pandas.read_csv(jobs_dir+'jobs_data_{0:}.csv'.format(this_state), encoding='utf-8', low_memory=False, index_col='rownumber')
#baltimore_df = df[df['w_geocode'].apply(lambda x: str(x)[0:5] == '24510')]
#baltimore_df.to_csv(baltimore_dir+'wac_jobs_df.csv', encoding='utf-8')
e = time.time()
g = g + (e-s)
print('Read {0:,.0f} rows in {1:,.0f} seconds!'.format(len(df), e-s))
df.head(2)

df.tail(3)

s = time.time()
jobs_meta_df = pandas.read_csv(data_dir+'lodes_wac_codebook.csv', encoding='utf-8', low_memory=False, index_col='varnum')
jobs_meta_df.to_csv(baltimore_dir+'wac_jobs_metadata.csv', encoding='utf-8')
e = time.time()
g = g + (e-s)
print('Wrote metadata for {0:,.0f} variables in {1:,.0f} seconds!'.format(len(jobs_meta_df), e-s))

# Delete the individual CSV files that we combined

In [22]:
jobs_files_list = sorted([x[-6:-4] for x in os.listdir(jobs_dir) if 'jobs_' in x])
print('Found {0:.0f} state jobs files!'.format(len(jobs_files_list)))
#pprint(jobs_files_list)
files_to_delete = sorted([jobs_dir+y for y in os.listdir(jobs_dir) if ('.csv' in y) and ('jobs_' not in y)])
print('Will delete {0:,.0f} files...'.format(len(files_to_delete)))
print('Deleting...')
for z in files_to_delete:
    os.remove(z)
print('Done!')

Found 51 state jobs files!
Will delete 33,620 files...
Deleting...
